In [1]:
# To get rid of those blocks of red warnings
import warnings
warnings.filterwarnings("ignore")

# Standard Imports
import numpy as np
from scipy import stats
import pandas as pd
from math import sqrt
import os

# Custom Module Imports
import env

# Modeling Imports
import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.feature_selection import f_regression 
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression, RFE

In [2]:
def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

def get_mallcustomer_data():
    '''
    Reads in all fields from the customers table in the mall_customers schema from data.codeup.com
    
    parameters: None
    
    returns: a single Pandas DataFrame with the index set to the primary customer_id field
    '''
    df = pd.read_sql('SELECT * FROM customers;', get_connection('mall_customers'))
    return df.set_index('customer_id')

In [3]:
df = get_mallcustomer_data()

In [4]:
df.head()

,gender,age,annual_income,spending_score
customer_id,,,,
1,Male,19,15,39
2,Male,21,15,81
3,Female,20,16,6
4,Female,23,16,77
5,Female,31,17,40


In [5]:
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    percnt_miss = num_missing / df.shape[0] * 100
    cols_missing = pd.DataFrame({'num_rows_missing' : num_missing,
                                'percent_rows_missing' : percnt_miss})
    return cols_missing

In [6]:
nulls_by_col(df)

,num_rows_missing,percent_rows_missing
gender,0,0.0
age,0,0.0
annual_income,0,0.0
spending_score,0,0.0


In [7]:
def nulls_by_row(df):
    num_missing = df.isnull().sum(axis=1)
    percnt_miss = num_missing / df.shape[1] * 100
    rows_missing = pd.DataFrame({'num_cols_missing' : num_missing,
                                'percent_cols_missing' : percnt_miss})
    return rows_missing

In [8]:
nulls_by_row(df)

,num_cols_missing,percent_cols_missing
customer_id,,
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0
5,0,0.0
...,...,...
196,0,0.0
197,0,0.0
198,0,0.0


In [9]:
def summarize_df(df):
    print('Shape:', df.shape)
    print('------------------------------\n')
    print('Info:\n')
    print(df.info())
    print('------------------------------\n')
    print('Describe:\n')
    print(df.describe().T)
    print('------------------------------\n')
    print('Nulls by Column:\n')
    print(nulls_by_col(df))
    print('------------------------------\n')
    print('Nulls by Row:\n')
    print(nulls_by_row(df))
    print('------------------------------\n')

In [10]:
summarize_df(df)

Shape: (200, 4)
------------------------------

Info:

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 1 to 200
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   gender          200 non-null    object
 1   age             200 non-null    int64 
 2   annual_income   200 non-null    int64 
 3   spending_score  200 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 7.8+ KB
None
------------------------------

Describe:

                count   mean        std   min    25%   50%   75%    max
age             200.0  38.85  13.969007  18.0  28.75  36.0  49.0   70.0
annual_income   200.0  60.56  26.264721  15.0  41.50  61.5  78.0  137.0
spending_score  200.0  50.20  25.823522   1.0  34.75  50.0  73.0   99.0
------------------------------

Nulls by Column:

                num_rows_missing  percent_rows_missing
gender                         0                   0.0
age                            

In [11]:
def remove_columns(df, cols_to_remove):  
    df = df.drop(columns=cols_to_remove)
    return df

def handle_missing_values(df, prop_required_column = .5, prop_required_row = .75):
    threshold = int(round(prop_required_column*len(df.index),0))
    df.dropna(axis=1, thresh=threshold, inplace=True)
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df

def data_prep(df, cols_to_remove=[], prop_required_column=.5, prop_required_row=.75):
    df = remove_columns(df, cols_to_remove)
    df = handle_missing_values(df, prop_required_column, prop_required_row)
    return df

In [12]:
df = data_prep(df, cols_to_remove=[],
    prop_required_column=.6, prop_required_row=.75)

In [13]:
def get_upper_outliers(s, k):
    '''
    Given a series and a cutoff value, k, returns the upper outliers for the
    series.

    The values returned will be either 0 (if the point is not an outlier), or a
    number that indicates how far away from the upper bound the observation is.
    '''
    q1, q3 = s.quantile([.25, .75])
    iqr = q3 - q1
    upper_bound = q3 + k * iqr
    return s.apply(lambda x: max([x - upper_bound, 0]))

def add_upper_outlier_columns(df, k):
    '''
    Add a column with the suffix _outliers for all the numeric columns
    in the given dataframe.
    '''
    # outlier_cols = {col + '_outliers': get_upper_outliers(df[col], k)
    #                 for col in df.select_dtypes('number')}
    # return df.assign(**outlier_cols)

    for col in df.select_dtypes('number'):
        df[col + '_outliers'] = get_upper_outliers(df[col], k)

    return df


In [14]:
add_upper_outlier_columns(df, k=1.5)

,gender,age,annual_income,spending_score,age_outliers,annual_income_outliers,spending_score_outliers
customer_id,,,,,,,
1,Male,19,15,39,0,0.00,0
2,Male,21,15,81,0,0.00,0
3,Female,20,16,6,0,0.00,0
4,Female,23,16,77,0,0.00,0
5,Female,31,17,40,0,0.00,0
...,...,...,...,...,...,...,...
196,Female,35,120,79,0,0.00,0
197,Female,45,126,28,0,0.00,0
198,Male,32,126,74,0,0.00,0


In [15]:
outlier_cols = [col for col in df if col.endswith('_outliers')]
for col in outlier_cols:
    print('~~~\n' + col)
    data = df[col][df[col] > 0]
    print(data.describe())

~~~
age_outliers
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: age_outliers, dtype: float64
~~~
annual_income_outliers
count    2.00
mean     4.25
std      0.00
min      4.25
25%      4.25
50%      4.25
75%      4.25
max      4.25
Name: annual_income_outliers, dtype: float64
~~~
spending_score_outliers
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: spending_score_outliers, dtype: float64


In [16]:
df.head()

,gender,age,annual_income,spending_score,age_outliers,annual_income_outliers,spending_score_outliers
customer_id,,,,,,,
1,Male,19,15,39,0,0.0,0
2,Male,21,15,81,0,0.0,0
3,Female,20,16,6,0,0.0,0
4,Female,23,16,77,0,0.0,0
5,Female,31,17,40,0,0.0,0


In [17]:
def split_data(df):
    train, test = train_test_split(df, test_size=.2, random_state=123)
    train, validate = train_test_split(train, test_size=.25, random_state=123)
    return train, validate, test

In [18]:
def scale_data(train, 
               validate, 
               test, 
               cols = ['annual_income', 'spending_score']):
    '''
    Scales the 3 data splits. 
    Takes in train, validate, and test data splits and returns their scaled counterparts.
    If return_scalar is True, the scaler object will be returned as well
    '''
    # make copies of our original data so we dont gronk up anything
    train_scaled = train.copy()
    validate_scaled = validate.copy()
    test_scaled = test.copy()
    #     make the thing
    scaler = sklearn.preprocessing.MinMaxScaler()
    #     fit the thing
    scaler.fit(train[cols])
    # applying the scaler:
    train_scaled[cols] = pd.DataFrame(scaler.transform(train[cols]),
                                                  columns=train[cols].columns.values).set_index([train.index.values])
                                                  
    validate_scaled[cols] = pd.DataFrame(scaler.transform(validate[cols]),
                                                  columns=validate[cols].columns.values).set_index([validate.index.values])
    
    test_scaled[cols] = pd.DataFrame(scaler.transform(test[cols]),
                                                 columns=test[cols].columns.values).set_index([test.index.values])
    return train_scaled, validate_scaled, test_scaled

In [30]:
def wrangle_mall():
    df = get_mallcustomer_data()
    summarize_df(df)
    df = data_prep(df, cols_to_remove=[], prop_required_column=.6, prop_required_row=.75)
    add_upper_outlier_columns(df, k=1.5)
    # creating dummy variables
    dummy_df = pd.get_dummies(df[['gender']], dummy_na=False)
    df = pd.concat([df, dummy_df], axis=1)
    train, validate, test = split_data(df)
    train_scaled, validate_scaled, test_scaled = scale_data(train, validate, test, cols = ['annual_income', 'spending_score'])
    return df, train, validate, test, train_scaled, validate_scaled, test_scaled

In [31]:
df, train, validate, test, train_scaled, validate_scaled, test_scaled = wrangle_mall()

Shape: (200, 4)
------------------------------

Info:

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 1 to 200
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   gender          200 non-null    object
 1   age             200 non-null    int64 
 2   annual_income   200 non-null    int64 
 3   spending_score  200 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 7.8+ KB
None
------------------------------

Describe:

                count   mean        std   min    25%   50%   75%    max
age             200.0  38.85  13.969007  18.0  28.75  36.0  49.0   70.0
annual_income   200.0  60.56  26.264721  15.0  41.50  61.5  78.0  137.0
spending_score  200.0  50.20  25.823522   1.0  34.75  50.0  73.0   99.0
------------------------------

Nulls by Column:

                num_rows_missing  percent_rows_missing
gender                         0                   0.0
age                            

In [32]:
df.head()

,gender,age,annual_income,spending_score,age_outliers,annual_income_outliers,spending_score_outliers,gender_Female,gender_Male
customer_id,,,,,,,,,
1,Male,19,15,39,0,0.0,0,0,1
2,Male,21,15,81,0,0.0,0,0,1
3,Female,20,16,6,0,0.0,0,1,0
4,Female,23,16,77,0,0.0,0,1,0
5,Female,31,17,40,0,0.0,0,1,0


In [33]:
train.head()

,gender,age,annual_income,spending_score,age_outliers,annual_income_outliers,spending_score_outliers,gender_Female,gender_Male
customer_id,,,,,,,,,
64,Female,54,47,59,0,0.0,0,1,0
49,Female,29,40,42,0,0.0,0,1,0
25,Female,54,28,14,0,0.0,0,1,0
137,Female,44,73,7,0,0.0,0,1,0
177,Male,58,88,15,0,0.0,0,0,1
